# Shop Assistant

In this notebook I built a chatbot for a business use. More specifically let's pretend that an online clothes shop wants to add a chatbot assistant that informs about items and the respective prices. I decided to hidden the choice of the model from the end user.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3


In [2]:
load_dotenv(override=True)

run = 'local'

if run == 'local':
    base_url = "http://localhost:11434/v1"
    api_key = 'ollama'
    MODEL = "llama3.2"
    openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
else:
    openai_api_key = os.getenv('OPENAI_API_KEY')
    if not openai_api_key:
        print("API key not found")
    MODEL = "gpt-4o-mini"
    openai = OpenAI()

In [4]:
conn = sqlite3.connect("./databases/shop.db")
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS items(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        category TEXT NOT NULL,
        price REAL NOT NULL,
        stock INTEGER NOT NULL       
    )
''')


In [5]:
cursor.execute("SELECT COUNT(*) FROM items")
if cursor.fetchone()[0] == 0:
    items = [
        ("airmax", "shoes", 180.99, 10),
        ("pull&bear", "jeans", 49.99, 25),
        ("pumplink", "t-shirt", 79.99, 50),
    ]
    cursor.executemany("INSERT INTO items (name, category, price, stock) VALUES (?, ?, ?, ?)", items)
    conn.commit()
conn.close()

In [7]:
def get_item_by_name(name):
    conn = sqlite3.connect("./databases/shop.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM items WHERE name = ?", (name,))
    row = cursor.fetchone()
    conn.close()
    price = None
    if row is not None:
        stock = row[4]
        if stock > 0: 
            price = row[3]
    return price

In [16]:
system_prompt = "You are a helpful assistant for an online shop. The shop is called ClothesAI and it sells jeans, t-shirts and shoes."
system_prompt += "Give a short and kind answer in no more than a sentence about prices and product availability."
system_prompt += "Be accurate and if you don't know the answer, say so."

In [17]:
search_database_function = {
    "name": "get_item_by_name",
    "description": "Get the price and the number of available items in stock. Call this whenever you need to know the item price and its availability, for example when a customer asks 'Do you have Levi's t-shirts?'", 
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type" : "string",
                "description": "The name of the the customer is looking for"
            }
        },
        "required": ["name"],
        "additionalParameters": False
    }
}

tools = [{"type": "function", "function": search_database_function}]

In [18]:
def handle_tool_call(message): 
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    item_name = arguments.get('name').strip().lower()
    price = get_item_by_name(item_name)
    response = {
        "role": "tool",
        "content": json.dumps({"name": item_name, "price": price}),
        "tool_call_id": tool_call.id
    }
    return response, item_name

In [21]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, item_name = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)    
    
    return response.choices[0].message.content


with gr.Blocks() as demo:
    gr.Markdown("## ClothesAI chatbot")
    with gr.Row():
        gr.ChatInterface(fn=chat, type="messages")
demo.launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
